# TFM : Aplicación de *Machine Learning* para la Gestión de Inventarios
> Erendira Teresa Navarro García

In [ ]:
# Python 3 environment Google Colab
import pandas as pd
import os
import csv
import datetime as datetime
import json 
import sklearn
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pylab as plab
from datetime import datetime
from dateutil.parser import parse
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from tabulate import tabulate
from pickle import dump

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import sklearn.metrics as metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
os.chdir("/content/drive/MyDrive/Files_TFM/RandomForest/")
print(os.getcwd())

/content/drive/MyDrive/Files_TFM/RandomForest


## Lectura de datos

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
df_tfm = pd.read_csv('/content/drive/MyDrive/Files_TFM/export_data_tfm.csv', parse_dates=['fecha'], date_parser=dateparse, dtype={'sku': str})
df_tfm = df_tfm.sort_values(by='fecha').set_index('fecha')

In [ ]:
df_tfm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51100 entries, 2019-01-02 to 2021-10-19
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   sku         51100 non-null  object 
 1   bolOpen     51100 non-null  int64  
 2   bolHoliday  51100 non-null  int64  
 3   udsVenta    51100 non-null  float64
 4   udsVentaO   51100 non-null  float64
 5   udsStock    32900 non-null  float64
 6   promo       51100 non-null  int64  
 7   udsVentaT   51100 non-null  float64
 8   label       51100 non-null  int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 3.9+ MB


## Preparación de los datos

In [ ]:
# Función para creación de nuevas variables - lag de 1 a 7 y diferencia en ventas del día anterior y del día previo
def features(dataset_t, columns_y):
  for i in range(1,8):
    dataset_t.loc[:,'lag'+str(i)] = dataset_t.loc[:,columns_y].shift(i)
    if i in [1,2]:
      dataset_t.loc[:,'diff'+str(i)] = dataset_t.loc[:,columns_y].diff(i)
  # Eliminar NAs
  dataset_t = dataset_t.dropna()
  return dataset_t

# Separar dataset y crear nuevas variables
def split_dataset(dataset, split_per, column_y):
  data_train, data_test = train_test_split(dataset, test_size=split_per, shuffle=False)
  data_train_total = features(data_train,column_y)
  data_test_total = features(data_test,column_y)
  y_train = data_train_total.loc[:,column_y]
  X_train = data_train_total.drop(columns=column_y)
  y_test = data_test_total.loc[:,column_y]
  X_test = data_test_total.drop(columns=column_y)
  return X_train, X_test, y_train, y_test

## Modelos

In [ ]:
def regression_results(y_true, y_pred, sku):
    # Métricas de precisión
    explained_var=explained_variance_score(y_true, y_pred)
    mae_value=mean_absolute_error(y_true, y_pred) 
    mse_value=mean_squared_error(y_true, y_pred) 
    mse_median=median_absolute_error(y_true, y_pred)
    rmse_value=np.sqrt(mse_value)
    r2_value = r2_score(y_true, y_pred)
    resultado_reg = pd.DataFrame([[str(sku),r2_value,explained_var,mae_value,mse_value,mse_median,rmse_value]],
                                 columns = ['sku','r2','explained_variance','MAE','MSE',
                                            'Median_MSE','RMSE'])
    return resultado_reg

In [ ]:
# Variables a utilizar en los modelos
columns_ok = ["bolOpen","promo"]
columns_to_scale  = ['lag'+str(i) for i in [1,2,7]] + ['diff'+str(i) for i in [1,2]]
y_to_scale  = ["udsVentaT"]
tscv = TimeSeriesSplit(n_splits=8)

In [ ]:
def models_over_sku(y_target, model, params, range_of_skus, name_of_model):
  df_trains = pd.DataFrame([],columns =["sku", "fecha","ventas_forecast","ventas"])
  df_tests = pd.DataFrame([],columns =["sku", "fecha","ventas_forecast","ventas"])
  df_models = pd.DataFrame([],columns =["sku", "modelo", "valor"])
  df_results_train = pd.DataFrame([],columns =['sku','r2','explained_variance','MAE','MSE',
                                            'Median_MSE','RMSE'])
  df_results_test = pd.DataFrame([],columns =['sku','r2','explained_variance','MAE','MSE',
                                            'Median_MSE','RMSE'])
  df_results_week = pd.DataFrame([],columns =['sku','r2','explained_variance','MAE','MSE',
                                            'Median_MSE','RMSE'])
  df_week = pd.DataFrame([],columns = ["week_of_year", "sku", "ventas_forecast",
                                       "stock_forecast", "ventas", "udsVenta", "udsStock"])
  for i in range_of_skus:
    dataset_sku = df_tfm[(df_tfm["sku"] == str(i))].drop(columns=["udsStock","label"])
    X_train, X_test, y_train, y_test = split_dataset(dataset_sku, 0.3, y_target)
    mm_scaler = MinMaxScaler(feature_range = (0,1))
    mm_y_scaler = MinMaxScaler(feature_range = (0,1))
    # Escala y
    y_train_array = mm_y_scaler.fit_transform(y_train.reset_index()[[y_target]]).ravel()
    y_test_array = mm_y_scaler.fit_transform(y_test.reset_index()[[y_target]]).ravel()
    # Escalar X
    X_tmp  = mm_scaler.fit_transform(X_train.reset_index()[columns_to_scale]) 
    X_train_array = np.concatenate([X_tmp , X_train.reset_index()[columns_ok].values], axis=1)
    X_tmp_t  = mm_scaler.fit_transform(X_test.reset_index()[columns_to_scale]) 
    X_test_array = np.concatenate([X_tmp_t , X_test.reset_index()[columns_ok].values], axis=1)
    # Modelos
    grf = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, n_jobs=-1, verbose=2)
    grf.fit(X_train_array, y_train_array)
    best_rf = grf.best_estimator_
    best_score = grf.score(X_train_array, y_train_array)
    df_m = pd.DataFrame([[str(i), best_rf, best_score]],columns =["sku", "modelo", "valor"])
    df_models = df_models.append(df_m, ignore_index = True)
    # Resultados train
    y_train_predict=pd.DataFrame(best_rf.predict(X_train_array)).rename(columns={0:'predicted_v'})
    y_test_predict=pd.DataFrame(best_rf.predict(X_test_array)).rename(columns={0:'predicted_v'})
    d_train = {'fecha': y_train.index.values, 
               'ventas_forecast': mm_y_scaler.inverse_transform(y_train_predict).ravel(),
               'ventas': y_train.reset_index().drop('fecha', axis=1).values.ravel()}
    df_train_= pd.DataFrame(data=d_train)
    df_train_["sku"] = str(i)
    df_trains = df_trains.append(df_train_[["sku", "fecha","ventas_forecast","ventas"]],ignore_index=True)
    df_train_=df_train_.set_index("fecha")
    
    d_test = {'fecha': y_test.index.values, 
              'ventas_forecast': mm_y_scaler.inverse_transform(y_test_predict).ravel(), 
              'ventas': y_test.reset_index().drop('fecha', axis=1).values.ravel()#,
              #'ventasT': X_test.udsVenta.values
              }
    df_test_ = pd.DataFrame(data=d_test)
    df_test_["sku"] = str(i)
    df_tests = df_tests.append(df_test_[["sku", "fecha","ventas_forecast","ventas"]],ignore_index=True)
    df_test_ = df_test_.set_index("fecha")
    
    # Resultados
    df_results_train = df_results_train.append(
        regression_results(y_train_array, 
                           y_train_predict,
                           i), ignore_index = True)
    df_results_test = df_results_test.append(
        regression_results(y_test_array, 
                           y_test_predict,
                           i), ignore_index = True)
    
    # save the model
    dump(best_rf, open('model_'+str(i)+'.pkl', 'wb'))
    # save the scaler
    dump(mm_scaler, open('minmax_scaler_'+str(i)+'.pkl', 'wb'))
    dump(mm_y_scaler, open('minmax_y_scaler_'+str(i)+'.pkl', 'wb'))
    
    plt.rcParams.update({'figure.figsize': (10,5)})
    plt.subplots()
    plt.plot(df_train_[["ventas_forecast"]], 'black', label = "Model", linewidth=0.5)
    plt.plot(df_train_[["ventas"]], 'orange', label = "Train", linewidth=0.5)
    plt.legend()
    plt.xlabel('fecha')
    plt.ylabel('ventas')
    plt.title("Forecast train result sku"+str(i))
    plt.tight_layout()
    plt.savefig("train_forecast_SKU_"+str(i)+".png", dpi=300, bbox_inches='tight')
    plt.close()

    plt.rcParams.update({'figure.figsize': (10,5)})
    plt.subplots()
    plt.plot(df_test_[["ventas_forecast"]], 'black', label = "Model", linewidth=0.5)
    plt.plot(df_test_[["ventas"]], 'blue', label = "Test", linewidth=0.5)
    plt.legend()
    plt.xlabel('fecha')
    plt.ylabel('ventas')
    plt.title("Forecast test result sku"+str(i))
    plt.tight_layout()
    plt.savefig("test_forecast_SKU_"+str(i)+".png", dpi=300, bbox_inches='tight')
    plt.close()

    # Complete data
    stock_validation = df_test_.merge(df_tfm.loc[(df_tfm["sku"] == str(i)),['udsVenta','udsStock']],
                                             how='inner',on='fecha')
    stock_validation["stock_forecast"] = stock_validation.ventas_forecast.apply(lambda x: x + (0.25*np.std(stock_validation["ventas_forecast"])) if x > 0 else 0)
    stock_validation['week_of_year'] = stock_validation.index.strftime('%W')
    
    df_w = stock_validation.groupby("week_of_year").agg(
        ventas_forecast = ('ventas_forecast','sum'),
        stock_forecast = ('stock_forecast','sum'),
        ventas = ('ventas','sum'),
        udsVenta = ('udsVenta','sum'),
        udsStock = ('udsStock','min')).reset_index()
    df_w["sku"] = str(i)
    df_w = df_w[["week_of_year", "sku", "ventas_forecast", "stock_forecast", "ventas", "udsVenta", "udsStock"]]

    plt.rcParams.update({'figure.figsize': (12,5)})
    plt.subplots()
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas_forecast"]], 'deepskyblue', label = "Ventas forecast", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["stock_forecast"]], 'lime', label = "Stock forecast", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas"]], 'red', label = "Ventas sin valores atípicos", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["udsVenta"]], 'black', label = "Ventas Original", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["udsStock"]], 'gray', label = "Stock", linewidth=0.8)
    plt.legend()
    plt.xlabel('semana')
    plt.ylabel('ventas')
    plt.title("Forecast stock and sales by week sku"+str(i))
    plt.tight_layout()
    plt.savefig("week_forecast_SKU_"+str(i)+".png", dpi=300, bbox_inches='tight')
    plt.close()

    df_week = df_week.append(df_w, ignore_index = True)
        
    df_results_week = df_results_week.append(
        regression_results(df_w.ventas.values, 
                           df_w.ventas_forecast.values,
                           i), ignore_index = True)

  return df_trains, df_tests, df_models, df_results_train, df_results_test, df_results_week, df_week

In [ ]:
model = RandomForestRegressor()
param_search = { 
    'n_estimators': [50,100,300,600,900],
    'max_depth' : [i for i in [5,25]],
    "max_features" : [i for i in range(1,8)]
}
y_info = "udsVentaT"
range_of_skus = range(1,51)
name_of_model = "Random Forest"

In [ ]:
[df_train, df_test, df_modelos, df_resultados_train, df_resultados_test, 
df_resultados_week, df_of_week] = models_over_sku(y_info, model, param_search, range_of_skus, name_of_model)

Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 8 folds for each of 70 candidates, totalling 560 fits
Fitting 

In [ ]:
df_resultados_train

,sku,r2,explained_variance,MAE,MSE,Median_MSE,RMSE
0,1,0.997213,0.997252,0.005310,0.000141,1.626126e-03,0.011893
1,2,0.996737,0.996803,0.007091,0.000191,2.673797e-03,0.013837
2,3,0.996341,0.996442,0.006580,0.000163,2.500000e-03,0.012785
3,4,0.997097,0.997112,0.006157,0.000174,1.666667e-03,0.013205
4,5,0.997826,0.997854,0.004604,0.000104,1.282051e-03,0.010207
5,6,0.995750,0.995775,0.006422,0.000229,1.666667e-03,0.015144
6,7,0.999065,0.999078,0.003385,0.000060,7.407407e-04,0.007755
7,8,0.997728,0.997756,0.005282,0.000155,1.065197e-03,0.012434
8,9,0.997192,0.997235,0.005014,0.000156,1.000000e-03,0.012483
9,10,0.998040,0.998072,0.003825,0.000103,7.560137e-04,0.010128


In [ ]:
df_resultados_test

,sku,r2,explained_variance,MAE,MSE,Median_MSE,RMSE
0,1,0.887075,0.951016,0.063284,0.006281,5.287838e-02,0.079252
1,2,0.965010,0.970586,0.028422,0.002204,1.465241e-02,0.046946
2,3,0.941434,0.948035,0.040677,0.003463,3.684659e-02,0.058851
3,4,0.949021,0.972812,0.036102,0.002411,3.500000e-02,0.049101
4,5,0.906263,0.939257,0.054575,0.007714,2.951049e-02,0.087829
5,6,0.918294,0.927880,0.050295,0.004421,4.333333e-02,0.066493
6,7,0.950092,0.974632,0.040077,0.003024,3.385522e-02,0.054987
7,8,0.969580,0.981905,0.022368,0.000990,1.821396e-02,0.031458
8,9,0.961729,0.963744,0.024878,0.001456,1.501515e-02,0.038162
9,10,0.975948,0.979684,0.021279,0.001239,8.599656e-03,0.035203


In [ ]:
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)
df_modelos.to_csv('modelos.csv', index=False)
df_resultados_train.to_csv('resultados_train.csv', index=False)
df_resultados_test.to_csv('resultados_test.csv', index=False)
df_resultados_week.to_csv('resultados_week.csv', index=False)
df_of_week.to_csv('info_of_week.csv', index=False)

In [ ]:
df_res_ventas_week = pd.DataFrame([], columns =['sku','r2','explained_variance','MAE','MSE','Median_MSE','RMSE'])
for i in range(1,51):
  df_w = df_of_week[df_of_week["sku"]==str(i)]
  df_res_ventas_week = df_res_ventas_week.append(
        regression_results(df_w.udsVenta.values, 
                           df_w.ventas_forecast.values,
                           i), ignore_index = True)

In [ ]:
df_res_ventas_week.to_csv('info_of_week.csv', index=False)

In [ ]:
df_res_ventas_week

,sku,r2,explained_variance,MAE,MSE,Median_MSE,RMSE
0,1,0.797996,0.976950,73.771926,5843.703505,78.968333,76.444120
1,2,0.953892,0.971759,25.758667,1529.023822,16.520000,39.102734
2,3,0.902273,0.907725,46.116667,3707.924260,39.810000,60.892727
3,4,0.924907,0.992415,29.517778,969.201271,29.580000,31.131998
4,5,0.851100,0.937668,51.648222,4585.214882,45.246667,67.714215
5,6,0.925123,0.955563,22.871556,740.755938,20.840000,27.216832
6,7,0.879297,0.978288,26.670370,867.318352,26.178333,29.450269
7,8,0.929380,0.987606,16.050173,306.921658,15.458889,17.519180
8,9,0.975375,0.984293,8.899852,113.839977,9.060000,10.669582
9,10,0.941106,0.951504,12.679593,328.562016,9.853333,18.126280


In [ ]:
for i in range(1,51):
    df_w = df_of_week[df_of_week["sku"]==str(i)]

    plt.rcParams.update({'figure.figsize': (12,5)})
    plt.subplots()
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas_forecast"]], 'deepskyblue', label = "Ventas forecast", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas"]], 'red', label = "Ventas sin valores atípicos", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["udsVenta"]], 'black', label = "Ventas Original", linewidth=1)
    plt.legend()
    plt.xlabel('semana')
    plt.ylabel('ventas')
    plt.title("Forecast stock and sales by week sku"+str(i))
    plt.tight_layout()
    plt.savefig("week_ventas_forecast_SKU_"+str(i)+".png", dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
for i in range(1,51):
    df_w = df_of_week[df_of_week["sku"]==str(i)]

    plt.rcParams.update({'figure.figsize': (12,5)})
    plt.subplots()
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas_forecast"]], 'deepskyblue', label = "Ventas forecast", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["stock_forecast"]], 'lime', label = "Stock forecast", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["ventas"]], 'red', label = "Ventas sin valores atípicos", linewidth=1)
    plt.plot(df_w.sort_values(by="week_of_year").set_index("week_of_year")[["udsVenta"]], 'black', label = "Ventas Original", linewidth=1)
    plt.legend()
    plt.xlabel('semana')
    plt.ylabel('ventas')
    plt.title("Forecast stock and sales by week sku"+str(i))
    plt.tight_layout()
    plt.savefig("week_forecast_bis_SKU_"+str(i)+".png", dpi=300, bbox_inches='tight')
    plt.close()